In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Data_Test.xlsx to Data_Test.xlsx
Saving Data_Train.xlsx to Data_Train.xlsx
Saving Sample_Submission.xlsx to Sample_Submission.xlsx


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
book_train = pd.read_excel('/content/Data_Train.xlsx')
book_test = pd.read_excel('/content/Data_Test.xlsx')

In [ ]:
book_train.head(10)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62
5,ChiRunning: A Revolutionary Approach to Effort...,Danny Dreyer,"Paperback,– 5 May 2009",4.5 out of 5 stars,8 customer reviews,The revised edition of the bestselling ChiRunn...,Healthy Living & Wellness (Books),Sports,900.00
6,Death on the Nile (Poirot),Agatha Christie,"Paperback,– 5 Oct 2017",4.4 out of 5 stars,72 customer reviews,Agatha Christie’s most exotic murder mystery\n...,"Crime, Thriller & Mystery (Books)","Crime, Thriller & Mystery",224.00
7,Yoga Your Home Practice Companion: A Complete ...,Sivananda Yoga Vedanta Centre,"Hardcover,– Import, 1 Mar 2018",4.7 out of 5 stars,16 customer reviews,"Achieve a healthy body, mental alertness, and ...",Sports Training & Coaching (Books),Sports,836.00
8,Karmayogi: A Biography of E. Sreedharan,M S Ashokan,"Paperback,– 15 Dec 2015",4.2 out of 5 stars,111 customer reviews,Karmayogi is the dramatic and inspiring story ...,Biographies & Autobiographies (Books),"Biographies, Diaries & True Accounts",130.00
9,"The Iron King (The Accursed Kings, Book 1)",Maurice Druon,"Paperback,– 26 Mar 2013",4.0 out of 5 stars,1 customer review,‘This is the original game of thrones’ George ...,Action & Adventure (Books),Action & Adventure,695.00


In [ ]:
book_test.head(10)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
0,The Complete Sherlock Holmes: 2 Boxes sets,Sir Arthur Conan Doyle,"Mass Market Paperback,– 1 Oct 1986",4.4 out of 5 stars,960 customer reviews,A collection of entire body of work of the She...,Short Stories (Books),"Crime, Thriller & Mystery"
1,Learn Docker - Fundamentals of Docker 18.x: Ev...,Gabriel N. Schenker,"Paperback,– Import, 26 Apr 2018",5.0 out of 5 stars,1 customer review,Enhance your software deployment workflow usin...,Operating Systems Textbooks,"Computing, Internet & Digital Media"
2,Big Girl,Danielle Steel,"Paperback,– 17 Mar 2011",5.0 out of 5 stars,4 customer reviews,"'Watch out, world. Here I come!'\nFor Victoria...",Romance (Books),Romance
3,Think Python: How to Think Like a Computer Sci...,Allen B. Downey,"Paperback,– 2016",4.1 out of 5 stars,11 customer reviews,"If you want to learn how to program, working w...",Programming & Software Development (Books),"Computing, Internet & Digital Media"
4,Oxford Word Skills: Advanced - Idioms & Phrasa...,Redman Gairns,"Paperback,– 26 Dec 2011",4.4 out of 5 stars,9 customer reviews,"Learn and practise the verbs, prepositions and...",Linguistics (Books),"Language, Linguistics & Writing"
5,Understanding a Photograph (Penguin Modern Cla...,John Berger,"Paperback,– 7 Nov 2013",5.0 out of 5 stars,2 customer reviews,John Berger's writings on photography are some...,"Arts History, Theory & Criticism (Books)","Arts, Film & Photography"
6,Dance Music Manual,Rick Snoman,"Paperback,– Import, 5 Feb 2019",5.0 out of 5 stars,1 customer review,"Dance Music Manual, aimed at the novice and se...",Music Books,"Computing, Internet & Digital Media"
7,A Clash of Kings (A Song of Ice and Fire),George R.R. Martin,"Paperback,– 29 Mar 2012",4.3 out of 5 stars,117 customer reviews,"The second volume of A Song of Ice and Fire, t...",Action & Adventure (Books),"Crime, Thriller & Mystery"
8,An Era of Darkness: The British Empire in India,Shashi Tharoor,"Hardcover,– 27 Oct 2016",4.4 out of 5 stars,550 customer reviews,"In 1930, the American historian and philosophe...",Asian History,Politics
9,Doing Justice: A Prosecutor’s Thoughts on Crim...,Preet Bharara,"Paperback,– 19 Mar 2019",4.4 out of 5 stars,6 customer reviews,A Guardian Pick for 2019\nThe People vs. Putin...,True Accounts (Books),Action & Adventure


In [ ]:
book_train.shape,book_test.shape

((6237, 9), (1560, 8))

In [ ]:
book_train.dtypes

Title            object
Author           object
Edition          object
Reviews          object
Ratings          object
Synopsis         object
Genre            object
BookCategory     object
Price           float64
dtype: object

In [ ]:
book_train.describe(include='all').head(2)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
count,6237,6237,6237,6237,6237,6237,6237,6237,6237.0
unique,5568,3679,3370,36,342,5549,345,11,NaN


In [ ]:
book_train.columns

Index(['Title', 'Author', 'Edition', 'Reviews', 'Ratings', 'Synopsis', 'Genre',
       'BookCategory', 'Price'],
      dtype='object')

In [ ]:
# no misssing value in book train 
book_train.isnull().sum()

Title           0
Author          0
Edition         0
Reviews         0
Ratings         0
Synopsis        0
Genre           0
BookCategory    0
Price           0
dtype: int64

In [ ]:
# no missing value in book test
book_test.isnull().sum()

Title           0
Author          0
Edition         0
Reviews         0
Ratings         0
Synopsis        0
Genre           0
BookCategory    0
dtype: int64

In [ ]:
# dropping unwanted columns i.e synopsis
book_train = book_train.drop(['Synopsis'],axis = 1)
book_test = book_test.drop(['Synopsis'],axis=1)

In [ ]:
book_train.head(2)

,Title,Author,Edition,Reviews,Ratings,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93


In [ ]:
book_test.head(2)

,Title,Author,Edition,Reviews,Ratings,Genre,BookCategory
0,The Complete Sherlock Holmes: 2 Boxes sets,Sir Arthur Conan Doyle,"Mass Market Paperback,– 1 Oct 1986",4.4 out of 5 stars,960 customer reviews,Short Stories (Books),"Crime, Thriller & Mystery"
1,Learn Docker - Fundamentals of Docker 18.x: Ev...,Gabriel N. Schenker,"Paperback,– Import, 26 Apr 2018",5.0 out of 5 stars,1 customer review,Operating Systems Textbooks,"Computing, Internet & Digital Media"


In [ ]:
# cleaning the edition columns 
def split_edition(data):  
  
  edition  = list(data)
  
  ed_type = [i.split(",– ")[0].strip().upper() for i in edition]
  
  edit_date = [i.split(",– ")[1].strip() for i in edition]
  
  m_y = [i.split()[-2:] for i in edit_date]
  
  
  for i in range(len(m_y)):
    if len(m_y[i]) == 1:
      m_y[i].insert(0,'NA')
      
  # Based on the given dataset below is the list of possible values for Months
  
  months =  ['Apr','Aug','Dec','Feb', 'Jan', 'Jul','Jun','Mar','May','NA','Nov','Oct','Sep']
  
  ed_month = [m_y[i][0].upper() if m_y[i][0] in months else 'NA' for i in range(len(m_y))]
  ed_year = [int(m_y[i][1].strip()) if m_y[i][1].isdigit() else 0 for i in range(len(m_y))]
  
  return ed_type, ed_month, ed_year

In [ ]:
#Identifying the maximum number of authors for a single book from the given datasets
authors_1 = list(book_train['Author'])
authors_2 = list(book_test['Author'])

authors_1.extend(authors_2)

authorslis = [i.split(",") for i in authors_1]

max = 1
for i in authorslis:
  if len(i) >= max:
    max = len(i)
print("Max. number of authors for a single boook = ",max)

for i in range(len(authorslis)):
  if len(authorslis[i]) == max:
    print(i)    
    
all_authors = [author.strip().upper() for listin in authorslis for author in listin]

Max. number of authors for a single boook =  7
7008


In [ ]:
# splitting into seven authors 
# A method to split the Author column in to 7 new columns
def split_authors(data):
  
  authors = list(data)
  
  A1 = []
  A2 = []
  A3 = []
  A4 = []
  A5 = []
  A6 = []
  A7 = []
  for i in authors:
    
    try :
      A1.append(i.split(',')[0].strip().upper())
    except :
      A1.append('NONE')
      
    try :
      A2.append(i.split(',')[1].strip().upper())
    except :
      A2.append('NONE')
        
    try :
      A3.append(i.split(',')[2].strip().upper())
    except :
      A3.append('NONE')
        
    try :
      A4.append(i.split(',')[3].strip().upper())
    except :
      A4.append('NONE')
        
    try :
      A5.append(i.split(',')[4].strip().upper())
    except :
      A5.append('NONE')
      
    try :
      A6.append(i.split(',')[5].strip().upper())
    except :
      A6.append('NONE')
     
    try :
      A7.append(i.split(',')[6].strip().upper())
    except :
      A7.append('NONE')

      
  return A1,A2,A3,A4,A5,A6,A7
  
all_authors.append('NONE')

In [ ]:
#Identifying the maximum number of Genres for a single book from the given datasets

genre_1 = list(book_train['Genre'])
genre_2 = list(book_test['Genre'])

genre_1.extend(genre_2)

genre_lis = [i.split(",") for i in genre_1]


max = 1
for i in genre_lis:
  if len(i) >= max:
    max = len(i)
print("Max. number of genres for a single boook = ",max)
      
all_genres = [genre.strip().upper() for listin in genre_lis for genre in listin]

Max. number of genres for a single boook =  2


In [ ]:
# A method to split the Genre column in to 2 new columns 

def split_genres(data):
  
  genres = list(data)
  
  G1 = []
  G2 = []
  
  for i in genres:
    
    try :
      G1.append(i.split(',')[0].strip().upper())
      
    except :
      G1.append('NONE')
      
    try :
      G2.append(i.split(',')[1].strip().upper())
    except :
      G2.append('NONE')


      
  return G1,G2
  
all_genres.append('NONE')

In [ ]:
#Identifying the maximum number of Categories for a single book from the given datasets

cat_1 = list(book_train['BookCategory'])
cat_2 = list(book_test['BookCategory'])

cat_1.extend(cat_2)

cat_lis = [i.split(",") for i in cat_1]


max = 1
for i in cat_lis:
  if len(i) >= max:
    max = len(i)
print("Max. number of Categories for a single boook = ",max)

all_categories = [cat.strip().upper() for listin in cat_lis for cat in listin]

Max. number of Categories for a single boook =  2


In [ ]:
# A method to split the Category column in to 2 new columsn 

def split_categories(data):
  
  cat = list(data)
  
  C1 = []
  C2 = []

  for i in cat:
    
    try :
      C1.append(i.split(',')[0].strip().upper())
    except :
      C1.append('NONE')
      
    try :
      C2.append(i.split(',')[1].strip().upper())
    except :
      C2.append('NONE')


      
  return C1,C2
  
all_categories.append('NONE')

In [ ]:
# A method to clean and restructure the datasets

import re

def restructure(data):
  
  #Cleaning Title Column
  titles = list(data['Title'])
  titles = [title.strip().upper() for title in titles]
  
  #Cleaning & Restructuring Author Column
  a1,a2,a3,a4,a5,a6,a7 = split_authors(data['Author']) 
  
  #Cleaning & Restructuring Edition Column
  ed_type, ed_month, ed_year = split_edition(data['Edition'])
  
  #Cleaning Ratings Column
  ratings = list(data['Reviews'])
  ratings = [float(re.sub(" out of 5 stars", "", i).strip()) for i in ratings]
  
  #Cleaning Reviews Column
  reviews = list(data['Ratings'])
  plu = ' customer reviews'
  reviews = [re.sub(" customer reviews", "", i) if plu in i else re.sub(" customer review", "", i) for i in reviews  ]
  reviews = [int(re.sub(",", "", i).strip()) for i in reviews ]
  

  #Cleaning & Restructuring Genre Column
  g1, g2 = split_genres(data['Genre'])
  
  #Cleaning & Restructuring BookCategory Column
  c1,c2 = split_categories(data['BookCategory'])

  # Forming the Structured dataset
  structured_data = pd.DataFrame({'Title': titles,
                                  'Author1': a1,
                                  'Author2': a2,
                                  'Author3': a3,
                                  'Author4': a4,
                                  'Author5': a5,
                                  'Author6': a6,
                                  'Author7': a7,
                                  'Edition_Type': ed_type,
                                  'Edition_Month': ed_month,
                                  'Edition_Year': ed_year,
                                  'Ratings': ratings,
                                  'Reviews': reviews,
                                  'Genre1': g1,
                                  'Genre2': g2,
                                  'Category1': c1,
                                  'Category2': c2
                                  
                               })
  
  return structured_data

In [ ]:
restructure(book_train).head(3)

,Title,Author1,Author2,Author3,Author4,Author5,Author6,Author7,Edition_Type,Edition_Month,Edition_Year,Ratings,Reviews,Genre1,Genre2,Category1,Category2
0,THE PRISONER'S GOLD (THE HUNTERS 3),CHRIS KUZNESKI,NONE,NONE,NONE,NONE,NONE,NONE,PAPERBACK,MAR,2016,4.0,8,ACTION & ADVENTURE (BOOKS),NONE,ACTION & ADVENTURE,NONE
1,GURU DUTT: A TRAGEDY IN THREE ACTS,ARUN KHOPKAR,NONE,NONE,NONE,NONE,NONE,NONE,PAPERBACK,NOV,2012,3.9,14,CINEMA & BROADCAST (BOOKS),NONE,BIOGRAPHIES,DIARIES & TRUE ACCOUNTS
2,LEVIATHAN (PENGUIN CLASSICS),THOMAS HOBBES,NONE,NONE,NONE,NONE,NONE,NONE,PAPERBACK,FEB,1982,4.8,6,INTERNATIONAL RELATIONS,NONE,HUMOUR,NONE


In [ ]:
X_train = restructure(book_train)

Y_train = book_train.iloc[:, -1].values

X_test = restructure(book_test)

In [ ]:
# encoding the data

In [ ]:
# A method for Finding Unique items for all columns
def unique_items(list1, list2):
  a = list1
  b = list2
  a.extend(b)
  return list(set(a))

In [ ]:
from sklearn.preprocessing import LabelEncoder

le_Title = LabelEncoder()
all_titles = unique_items(list(X_train.Title),list(X_test.Title))
le_Title.fit(all_titles)

le_Edition_Type = LabelEncoder()
all_etypes = unique_items(list(X_train.Edition_Type),list(X_test.Edition_Type))
le_Edition_Type.fit(all_etypes)


le_Edition_Month = LabelEncoder()
all_em = unique_items(list(X_train.Edition_Month),list(X_test.Edition_Month))
le_Edition_Month.fit(all_em)

le_Author = LabelEncoder()
all_Authors = list(set(all_authors))
le_Author.fit(all_Authors)

le_Genre = LabelEncoder()
all_Genres = list(set(all_genres))
le_Genre.fit(all_Genres)

le_Category = LabelEncoder()
all_Categories = list(set(all_categories))
le_Category.fit(all_Categories)

LabelEncoder()

In [ ]:
X_train['Title'] = le_Title.transform(X_train['Title'])

X_train['Edition_Type'] = le_Edition_Type.transform(X_train['Edition_Type'])



X_train['Edition_Month'] = le_Edition_Month.transform(X_train['Edition_Month'])

X_train['Author1'] = le_Author.transform(X_train['Author1'])
X_train['Author2'] = le_Author.transform(X_train['Author2'])
X_train['Author3'] = le_Author.transform(X_train['Author3'])
X_train['Author4'] = le_Author.transform(X_train['Author4'])
X_train['Author5'] = le_Author.transform(X_train['Author5'])
X_train['Author6'] = le_Author.transform(X_train['Author6'])
X_train['Author7'] = le_Author.transform(X_train['Author7'])


X_train['Genre1'] = le_Genre.transform(X_train['Genre1'])
X_train['Genre2'] = le_Genre.transform(X_train['Genre2'])


X_train['Category1'] = le_Category.transform(X_train['Category1'])
X_train['Category2'] = le_Category.transform(X_train['Category2'])

In [ ]:
X_train.head()

,Title,Author1,Author2,Author3,Author4,Author5,Author6,Author7,Edition_Type,Edition_Month,Edition_Year,Ratings,Reviews,Genre1,Genre2,Category1,Category2
0,5802,797,3073,3073,3073,3073,3073,3073,13,7,2016,4.0,8,0,267,0,12
1,2120,391,3073,3073,3073,3073,3073,3073,13,10,2012,3.9,14,80,267,2,6
2,2984,4353,3073,3073,3073,3073,3073,3073,13,3,1982,4.8,6,211,267,8,12
3,189,78,3073,3073,3073,3073,3073,3073,13,11,2017,4.1,13,98,267,5,16
4,2987,1221,3073,3073,3073,3073,3073,3073,8,11,2006,5.0,1,284,267,1,7


In [ ]:
X_test['Title'] = le_Title.transform(X_test['Title'])

X_test['Edition_Type'] = le_Edition_Type.transform(X_test['Edition_Type'])

X_test['Edition_Month'] = le_Edition_Month.transform(X_test['Edition_Month'])

X_test['Author1'] = le_Author.transform(X_test['Author1'])
X_test['Author2'] = le_Author.transform(X_test['Author2'])
X_test['Author3'] = le_Author.transform(X_test['Author3'])
X_test['Author4'] = le_Author.transform(X_test['Author4'])
X_test['Author5'] = le_Author.transform(X_test['Author5'])
X_test['Author6'] = le_Author.transform(X_test['Author6'])
X_test['Author7'] = le_Author.transform(X_test['Author7'])


X_test['Genre1'] = le_Genre.transform(X_test['Genre1'])
X_test['Genre2'] = le_Genre.transform(X_test['Genre2'])


X_test['Category1'] = le_Category.transform(X_test['Category1'])
X_test['Category2'] = le_Category.transform(X_test['Category2'])

In [ ]:
X_test.head()

,Title,Author1,Author2,Author3,Author4,Author5,Author6,Author7,Edition_Type,Edition_Month,Edition_Year,Ratings,Reviews,Genre1,Genre2,Category1,Category2
0,5082,4058,3073,3073,3073,3073,3073,3073,12,11,1986,4.4,960,324,267,5,16
1,2906,1401,3073,3073,3073,3073,3073,3073,13,0,2018,5.0,1,273,267,4,9
2,751,949,3073,3073,3073,3073,3073,3073,13,7,2011,5.0,4,314,267,14,12
3,6232,169,3073,3073,3073,3073,3073,3073,13,9,2016,4.1,11,295,267,4,9
4,3790,3505,3073,3073,3073,3073,3073,3073,13,2,2011,4.4,9,235,267,10,11


In [ ]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)

X_test = sc.transform(X_test)

#Reshaping ti fit the scaler
Y_train = Y_train.reshape((len(Y_train), 1)) 

Y_train = sc.fit_transform(Y_train)

#Restoring the original shape after scaling
Y_train = Y_train.ravel()

In [ ]:
X_train.shape,Y_train.shape

((6237, 17), (6237,))

In [ ]:
X=X_train
y=Y_train

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score

regressors = {
    'Decision Tree' : DecisionTreeRegressor(),
    'Random Forest' : RandomForestRegressor(),
    'Support Vector Machines' : SVR(),
    'K-nearest Neighbors' : KNeighborsRegressor(),
    'XGBoost' : XGBRegressor(objective='reg:squarederror', max_depth=6, learning_rate=0.1, n_estimators=100, booster = 'gbtree', n_jobs = -1,random_state = 1)
}
results=pd.DataFrame(columns=['MAE','MSE','R2-score'])
for method,func in regressors.items():
    func.fit(X_train,y_train)
    pred = func.predict(X_test)
    results.loc[method]= [mean_absolute_error(y_test,pred),
                          mean_squared_error(y_test,pred),
                          r2_score(y_test,pred)
                         ]
results

,MAE,MSE,R2-score
Decision Tree,0.590936,2.177938,-1.499646
Random Forest,0.413048,0.681313,0.218049
Support Vector Machines,0.389484,0.800094,0.081722
K-nearest Neighbors,0.479680,0.858319,0.014896
XGBoost,0.415508,0.729778,0.162425


In [ ]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(X_train, y_train.ravel())


DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=None,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=0, splitter='best')

In [ ]:
# Predicting for test set
y_pred_decision = sc.inverse_transform(regressor.predict(X_test))

In [ ]:
# Saving the predictions in excel file

solution = pd.DataFrame(y_pred_decision, columns = ['Price'])
solution.to_excel('Predict_Book_Price.xlsx', index = False)

In [ ]:
solution

,Price
0,360.0
1,415.0
2,2798.0
3,184.0
4,335.0
...,...
931,135.0
932,460.0
933,2166.0
934,299.0


In [ ]:
from sklearn.model_selection import train_test_split

train_x, val_x, train_y, val_y = train_test_split(X, y, test_size = 0.1, random_state = 123)

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(val_x.shape)
print(val_y.shape)

(5613, 17)
(5613,)
(624, 17)
(624,)


In [ ]:
from xgboost import XGBRegressor
import numpy as np

xgb=XGBRegressor( objective='reg:squarederror', max_depth=6, learning_rate=0.1, n_estimators=100, booster = 'gbtree', n_jobs = -1,random_state = 1)
xgb.fit(train_x,train_y)

y_pred = sc.inverse_transform(xgb.predict(val_x))
y_true = sc.inverse_transform(val_y)

error = np.square(np.log10(y_pred +1) - np.log10(y_true +1)).mean() ** 0.5
score = 1 - error

print("RMLSE Score = ", score)

RMLSE Score =  0.7163958922112829


In [ ]:
# Fitting the complete training set (inclusing val_x and val_y)
xgb.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=6, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=-1, nthread=None, objective='reg:squarederror',
             random_state=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [ ]:
# Predicting for test set
y_pred_xgb = sc.inverse_transform(xgb.predict(X_test))

In [ ]:
# Saving the predictions in excel file

solution1 = pd.DataFrame(y_pred_xgb, columns = ['Price'])
solution1.to_excel('Book_Price.xlsx', index = False)

In [ ]:
solution1

,Price
0,641.898743
1,531.566040
2,1546.009277
3,322.143768
4,514.361023
...,...
931,648.140747
932,667.406799
933,8739.594727
934,665.965027
